In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display
import statsmodels.api as sm


In [2]:
df = pd.read_csv("Άσκηση ΜΚ-04 - Επικύρωση Μεθόδου T-3 Ενδιάμεση Πιστότητα.csv", header=1)
display.Markdown(df.to_markdown())

|    | Repeat                              |   Analyst |   Sample_1 |   Sample_2 |
|---:|:------------------------------------|----------:|-----------:|-----------:|
|  0 | 1                                   |         1 |      495.6 |     1013.5 |
|  1 | 2                                   |         1 |      512   |     1015.6 |
|  2 | 3                                   |         1 |      511.6 |     1008.9 |
|  3 | 4                                   |         1 |      527.2 |     1019.9 |
|  4 | 5                                   |         1 |      509.3 |     1014.6 |
|  5 | 6                                   |         1 |      523.7 |     1020.2 |
|  6 | 1                                   |         2 |      500.9 |      997.5 |
|  7 | 2                                   |         2 |      502.9 |     1027.8 |
|  8 | 3                                   |         2 |      507.4 |      987.8 |
|  9 | 4                                   |         2 |      513   |      886.1 |
| 10 | 5                                   |         2 |      501.9 |      967.2 |
| 11 | 6                                   |         2 |      509.6 |     1008.3 |
| 12 | 1                                   |         3 |      478.3 |      994.4 |
| 13 | 2                                   |         3 |      514.8 |     1012.2 |
| 14 | 3                                   |         3 |      511   |     1027.2 |
| 15 | 4                                   |         3 |      508.3 |     1022.5 |
| 16 | 5                                   |         3 |      517.1 |     1022   |
| 17 | 6                                   |         3 |      522.7 |     1028.1 |
| 18 | Grand Average                       |       nan |      nan   |      nan   |
| 19 | Standard Deviation (SDR)            |       nan |      nan   |      nan   |
| 20 | Relative Standard Deviation (%RSDR) |       nan |      nan   |      nan   |

In [3]:
def Q_test(ser, ci =95):
    "Handle variable titles. This needs to be a class. Type check errors"
    Qtable = pd.DataFrame(data = {
        'Number of values': ['3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'],
        '90%':[0.941,0.765,0.642,0.560,0.507,0.468,0.437,0.412,0.3312,0.3170,0.3045,0.2938,0.2848,0.2765,0.2691,0.2626,0.2564,0.2511],
        '95%':[0.970,0.829,0.710,0.625,0.568,0.526,0.493,0.466,0.3922,0.3755,0.3615,0.3496,0.3389,0.3293,0.3208,0.3135,0.3068,0.3005],
        '99%':[0.994,0.926,0.821,0.740,0.680,0.634,0.598,0.568,0.5028,0.4831,0.4664,0.4517,0.4385,0.4268,0.4166,0.4081,0.4002,0.3922],
    })
    if ci not in (90,95,99):
        raise ValueError('Confidence interval must be 90,95,99 as an integer')
    Qtheo = Qtable[Qtable['Number of values'] == str(len(ser))][str(ci)+'%'].iloc[0]
    ser = ser.sort_values()
    rang = abs(ser.iloc[len(ser)-1]-ser.iloc[0])
    gap_min = abs(ser.iloc[1]-ser.iloc[0])
    gap_max = abs(ser.iloc[len(ser)-1]-ser.iloc[len(ser)-2])
    if gap_min/rang>Qtheo:
        return (True, ser.iloc[0])
    elif gap_max>Qtheo:
        return (True, ser.iloc[len(ser)-1])
    else:
        return (False,None)

In [4]:
data = df[df["Analyst"] == 1]["Sample_1"]
data = pd.DataFrame(data = {"A1S1": df[df["Analyst"] == 1]["Sample_1"]})
i = False
for samp in [1,2]:
    for analyst in df["Analyst"].dropna().unique():
        if i:
            data = data.join(pd.DataFrame(data = {f"A{int(analyst)}S{samp}": list(df[df["Analyst"] == 1]["Sample_1"])} ) )
        else:
            i = True
for i in range(3):
    data=data.append(pd.Series([None for x in range(6)]),ignore_index=True)
data

,A1S1,A2S1,A3S1,A1S2,A2S2,A3S2,0,1,2,3,4,5
0,495.6,495.6,495.6,495.6,495.6,495.6,NaN,NaN,NaN,NaN,NaN,NaN
1,512.0,512.0,512.0,512.0,512.0,512.0,NaN,NaN,NaN,NaN,NaN,NaN
2,511.6,511.6,511.6,511.6,511.6,511.6,NaN,NaN,NaN,NaN,NaN,NaN
3,527.2,527.2,527.2,527.2,527.2,527.2,NaN,NaN,NaN,NaN,NaN,NaN
4,509.3,509.3,509.3,509.3,509.3,509.3,NaN,NaN,NaN,NaN,NaN,NaN
5,523.7,523.7,523.7,523.7,523.7,523.7,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None
